In [1]:
PROJECT = "qwiklabs-gcp-ml-8d503f4cdc07"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-8d503f4cdc07"  # Replace with your BUCKET

REGION = "us-central1"            # Choose an available region for Cloud CAIP
TFVERSION = "1.14"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [4]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split
print(tf.__version__)

1.15.0


In [7]:
%%bash
gsutil ls gs://${BUCKET}/data/*.csv

gs://qwiklabs-gcp-ml-8d503f4cdc07/data/creditcard.csv


In [10]:
gcs_data = pd.read_csv('gs://{}/data/creditcard.csv'.format(BUCKET))

In [11]:
gcs_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [15]:
!pwd

/home/jupyter/google-asl-study/final_mini_project/kmlee


In [12]:
## data setting
# setting up testing and training sets
df_train, df_test = train_test_split(gcs_data, test_size=0.2, random_state=27)

print('df_train :', df_train.shape)
print('df_test :', df_test.shape)
print('df_train Class: \n', df_train.Class.value_counts())

df_train : (227845, 31)
df_test : (56962, 31)
df_train Class: 
 0    227457
1       388
Name: Class, dtype: int64


In [13]:
df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=27)
print('df_train :', df_train.shape)
print('df_valid :', df_valid.shape)
print('df_test :', df_test.shape)

df_train : (182276, 31)
df_valid : (45569, 31)
df_test : (56962, 31)


In [14]:
df_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
26120,33919.0,-2.388049,-2.490423,1.888346,-1.313157,0.335402,-0.743883,0.576642,-0.071353,1.228283,...,0.569097,0.908894,0.826554,0.089925,0.499400,-0.707443,-0.183655,-0.052248,383.35,0
9192,13161.0,1.225984,0.212700,-0.157300,1.082289,0.840563,1.184308,-0.127604,0.139463,1.385294,...,-0.303960,-0.426814,-0.290104,-1.754287,0.847065,-0.263307,0.007218,-0.015656,19.08,0
230012,146147.0,-2.188785,-0.958016,0.769310,-3.102286,-0.242737,0.071822,-0.486386,0.269428,-1.919930,...,-0.252410,-0.148452,-0.605173,0.213868,0.737452,-0.193485,-0.152167,0.006639,79.00,0
240894,150796.0,2.105378,0.095780,-2.263791,0.091285,0.882610,-0.338245,0.099070,-0.112726,0.483264,...,0.157169,0.593088,-0.190231,-1.519934,0.379331,-0.028933,-0.007032,-0.056937,1.80,0
28768,35150.0,-0.328728,-0.476626,1.429567,-0.900624,0.067534,2.167218,-0.040161,0.497778,-1.119866,...,0.210792,0.801463,0.142101,-1.346529,-0.619436,-0.095093,0.029224,-0.106220,96.62,0


In [15]:
df_valid.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
269400,163647.0,0.129017,1.896071,-1.909277,-0.085467,1.516371,-2.947193,1.922066,-0.649116,-0.797983,...,0.046164,0.272604,-0.286842,0.621032,0.080475,0.108701,0.046224,0.123110,0.30,0
140345,83671.0,-2.810614,-1.255934,0.534252,1.833196,1.266633,-0.336898,-0.524892,0.566502,-0.583604,...,0.029318,0.150226,-0.861510,-0.163730,-0.210010,-0.245811,0.605932,-0.486416,108.86,0
234612,148046.0,1.632977,-0.650973,-0.700894,1.498131,-0.548833,-0.505637,-0.043126,-0.114222,0.870073,...,0.142117,0.172234,0.036681,-0.121723,-0.148125,-0.681985,0.008653,-0.007497,179.00,0
69669,53524.0,-1.244625,-0.862845,0.902662,-0.220987,-1.363494,0.435640,0.255022,0.640365,-1.588047,...,0.227404,0.313352,0.719519,-0.067816,-0.091712,-0.451338,-0.153241,-0.125369,310.96,0
80277,58411.0,1.216248,0.217858,-0.185325,0.771201,0.106515,0.003741,-0.162653,0.071036,0.308806,...,-0.177230,-0.274791,-0.216183,-0.543854,0.644877,0.443618,-0.000127,0.025547,12.31,0


In [16]:
df_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
47747,43343.0,1.209971,-0.158081,0.685984,-0.019681,-0.874075,-0.775662,-0.352771,0.015835,0.545579,...,-0.096262,-0.294659,0.177728,0.418381,-0.039814,0.918010,-0.061961,0.007276,5.37,0
225706,144357.0,0.107350,0.718755,-0.059117,-0.821294,0.716260,-1.189281,1.338169,-0.395374,-0.185327,...,-0.043964,-0.030030,-0.042275,-0.078641,-0.157494,-0.150633,0.019411,0.036256,31.00,0
255432,157223.0,-0.992778,1.374399,-1.681843,-0.123910,0.711935,-1.394937,2.114597,-0.066849,-0.815940,...,0.196423,0.220383,0.131703,-0.358277,0.402663,-0.552094,-0.015239,0.170879,231.05,0
202435,134327.0,2.084975,-1.678690,-0.492561,-1.592379,-1.541775,-0.284969,-1.310430,-0.051357,-1.320674,...,-0.315913,-0.614923,0.334403,-0.383453,-0.604302,-0.494455,0.023015,-0.032347,95.94,0
119363,75422.0,1.176209,0.183813,0.514542,0.450080,-0.197021,-0.187294,-0.091779,0.056852,-0.280994,...,-0.173952,-0.483480,0.149046,0.023991,0.129106,0.105646,-0.013529,0.006163,0.99,0


In [17]:
df_test.to_csv('test.csv', index=False)

In [18]:
df_valid.to_csv('valid.csv', index=False)

In [19]:
df_train.to_csv('train.csv', index=False)

In [20]:
df_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
47747,43343.0,1.209971,-0.158081,0.685984,-0.019681,-0.874075,-0.775662,-0.352771,0.015835,0.545579,...,-0.096262,-0.294659,0.177728,0.418381,-0.039814,0.918010,-0.061961,0.007276,5.37,0
225706,144357.0,0.107350,0.718755,-0.059117,-0.821294,0.716260,-1.189281,1.338169,-0.395374,-0.185327,...,-0.043964,-0.030030,-0.042275,-0.078641,-0.157494,-0.150633,0.019411,0.036256,31.00,0
255432,157223.0,-0.992778,1.374399,-1.681843,-0.123910,0.711935,-1.394937,2.114597,-0.066849,-0.815940,...,0.196423,0.220383,0.131703,-0.358277,0.402663,-0.552094,-0.015239,0.170879,231.05,0
202435,134327.0,2.084975,-1.678690,-0.492561,-1.592379,-1.541775,-0.284969,-1.310430,-0.051357,-1.320674,...,-0.315913,-0.614923,0.334403,-0.383453,-0.604302,-0.494455,0.023015,-0.032347,95.94,0
119363,75422.0,1.176209,0.183813,0.514542,0.450080,-0.197021,-0.187294,-0.091779,0.056852,-0.280994,...,-0.173952,-0.483480,0.149046,0.023991,0.129106,0.105646,-0.013529,0.006163,0.99,0


In [21]:
!pwd

/home/jupyter/google-asl-study/final_mini_project/kmlee


In [22]:
FEATURE_LEN = 31
os.environ['FEATURE_LEN'] = str(FEATURE_LEN)

In [ ]:
%%bash
DATADIR=$(pwd)
OUTDIR=$(pwd)/trained/miniproj
rm -rf $OUTDIR
gcloud ai-platform local train \
   --module-name=miniproj.trainer.task \
   --package-path=${PWD}/miniproj.trainer \
   -- \
   --train_data_path="${DATADIR}/train.csv" \
   --eval_data_path="${DATADIR}/valid.csv"  \
   --output_dir=${OUTDIR} \
   --model=linear --train_steps=10 --feature_length=$FEATURE_LEN

In [27]:
%%bash
mkdir -p miniproj/trainer
touch miniproj/__init__.py
touch miniproj/trainer/__init__.py

In [24]:
%%writefile miniproj/trainer/task.py
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--feature_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing miniproj/trainer/task.py


In [25]:
%%writefile miniproj/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

LABLE_COL = 'height'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
FEATURE_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global FEATURE_LEN, DEFAULTS, N_INPUTS
    FEATURE_LEN = hparams['feature_length']
    DEFAULTS = [[0.0] for x in range(0, FEATURE_LEN)]
    N_INPUTS = FEATURE_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[LABLE_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[LABLE_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[LABLE_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[LABLE_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[LABLE_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[LABLE_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {LABLE_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        
        dataset = dataset.skip(1).map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        LABLE_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[LABLE_COL] = tf.squeeze(features[LABLE_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[LABLE_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[LABLE_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def data_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=data_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_secs=
                                           hparams['min_eval_frequency']),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      start_delay_secs=hparams['eval_delay_secs'],
                                      throttle_secs=hparams['min_eval_frequency'])
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


Writing miniproj/trainer/model.py
